## Final Projects

_course: quantum cryptography for beginners
<br>date: 22 december 2022
<br>author: burton rosenberg_



### Teleportation and Quantum One Time Pad

The class investigated a possible relationship between the one time pad, quantum teleportation, and super dense coding. The intuitive relationship is that there can be no superluminal communication, yet quantum teleportation can transport a state instantaneously. However, in order to extract that information, classical communication and classical results need to be transmitted. In fact, the teleported qubit arrives information-theoretically sealed, until the 2 classcal bits arrive to unseal the qubit.

The became a project for Andrew, who attempted to extract information from the teleported qubit, without reference to the classcial bits. From the field of classical cryptography he set an Adversarial Indistinguishability Game and also from classical cryptography gave a general inner product model for information read-out.

The AIG is that, given two qubits of the adversary's choosing, does the receiver, without help of the two classical qubits, have an algorithm (with unbounded computation) for guessing which of the qubits was input with success probability different than 1/2. 

Along the way, he explored how an inner product can be computed in a quantum circuit. 


I created a model circuit of Andrew's ideas, found in the following panel. His own work is found [here](https://github.com/AjSiciliano/Analysis-of-Superluminal-Communication-Using-Quantum-Teleportation).

The use of quantum calculated inner produces is also featured in a toy linear least squares optimization as suggested in the [IBM worksheet](https://qiskit.org/textbook/ch-demos/variational-quantum-regression.html). Andrew's rendition is [here](https://github.com/AjSiciliano/Variational-Quantum-Regression-Using-PennyLane).

In [1]:
#
# Program to explore using the inner product
# on a teleported state, presumably to pass information
#
# author: bjr
# last-update:
#		8 dec 2022: 
#
# Suppose one of two vectors were chosen, phi1 and phi2, and
# teleported. The teleported vectors were then inner producted
# against a fixed vector. Is it possible to tell which of the
# two vectors was chosen? 
#

from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from numpy import sqrt
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.circuit import Parameter
import math
import cmath
import numpy as np


class TeleportInnerProduct:
    
    def __init__(self,alpha,beta):
        self.qc = QuantumCircuit(5,1)
        self.theta = Parameter('theta')
        self.phi = Parameter('phi')
        self.sim = Aer.get_backend('aer_simulator')
        r = math.sqrt(abs(alpha)**2+abs(beta)**2)
        self.alpha = alpha/r
        self.beta = beta/r
        self.n = 5
        self.m = 1
        
    def teleportation_circuit(self,k):
        self.qc.h(k+1)
        self.qc.cx(k+1,k+2)
        self.qc.cx(k,k+1)
        self.qc.h(k)
        self.qc.barrier(range(self.n))
        
    def innerproduct_circuit(self,k):
        self.qc.h(k+2)
        self.qc.cswap(k+2,k+1,k)
        self.qc.h(k+2)
        self.qc.barrier(range(self.n))
        
    def test_phi(self,alpha,beta):
        self.qc.ry(self.theta,0)
        self.qc.rx(self.phi,0)
        self.qc.initialize([alpha,beta], 3)
        self.qc.barrier(range(self.n))
        
    def make_circuit(self):
        self.test_phi(self.alpha,self.beta)
        self.teleportation_circuit(0)
        self.innerproduct_circuit(2)
        self.qc.measure(4,0)
        
    def run(self,theta,phi,draw=False):
        qc = self.qc.bind_parameters({self.theta: theta, self.phi:phi})
        if draw:
            print(qc.draw())
        job = self.sim.run(transpile(qc, self.sim))
        counts = job.result().get_counts()
        return { k: counts[k] for k in sorted(counts,key=counts.get,reverse=True)}
        
def test(alpha,beta):     
    tip = TeleportInnerProduct(alpha,beta)
    tip.make_circuit()
    print(tip.qc.draw())
    print(f'test vector: ({alpha:.4f},{beta:.4f})')
    print(f'pi/16\tpi/32\tcounts\n--------------------------------------')
    for i in range(16):
        print(f'{i}\t{i}\t{tip.run(i*math.pi/16,i*math.pi/64)}')
        
test(1.0,0.0)
test(1.0+0.25j,1.0j)


        ┌───────────┐   ┌─────────┐ ░                ┌───┐ ░               ░ »
q_0: ───┤ Ry(theta) ├───┤ Rx(phi) ├─░─────────────■──┤ H ├─░───────────────░─»
        └───────────┘   └─────────┘ ░ ┌───┐     ┌─┴─┐└───┘ ░               ░ »
q_1: ───────────────────────────────░─┤ H ├──■──┤ X ├──────░───────────────░─»
                                    ░ └───┘┌─┴─┐└───┘      ░               ░ »
q_2: ───────────────────────────────░──────┤ X ├───────────░───────X───────░─»
     ┌─────────────────┐            ░      └───┘           ░       │       ░ »
q_3: ┤ Initialize(1,0) ├────────────░──────────────────────░───────X───────░─»
     └─────────────────┘            ░                      ░ ┌───┐ │ ┌───┐ ░ »
q_4: ───────────────────────────────░──────────────────────░─┤ H ├─■─┤ H ├─░─»
                                    ░                      ░ └───┘   └───┘ ░ »
c: 1/════════════════════════════════════════════════════════════════════════»
                                                    

## 3-SAT

Maki used Grover Algorithm to explore solutions to 3-SAT. The surprising elements here are 

- was when there are several solutions, 
- or when there are plentiful solutions, 
- or when actually this is a question of non-tautology (find a non-satisfying assignment)
- or the cases of contraction or tautology.

The Grover Algorithm will randomly find a solution. And in the case of non-tautology, it will find 
non-satisfying assignments. In some cases it seems to get off to a start towards on direction or
another. 

Here results are provided [here](https://github.com/csc-courses/csc685/blob/master/exercises/Satisfiability_problem.pdf).


## Superdense coding

In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from numpy import sqrt
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator

sim = Aer.get_backend('aer_simulator')

def super_dense(i,j):
    qc = QuantumCircuit(2)

    # Eve
    qc.h(0)
    qc.cx(0,1)
    qc.barrier(0,1)
    
    # Alice
    if i:
        qc.x(0)
    if j:
        qc.z(0)
        #qc.z(1)
    qc.barrier(0,1)
    
    # Bob
    qc.cx(0,1)
    qc.h(0)

    qc.measure_all()
    return qc

for i in range(2):
    for j in range(2):
        qc = super_dense(i,j)
        result = sim.run(qc).result()
        print(i,j)
        print(result.get_counts())
        print(qc.draw())

0 0
{'00': 1024}
        ┌───┐      ░  ░      ┌───┐ ░ ┌─┐   
   q_0: ┤ H ├──■───░──░───■──┤ H ├─░─┤M├───
        └───┘┌─┴─┐ ░  ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──░─┤ X ├──────░──╫─┤M├
             └───┘ ░  ░ └───┘      ░  ║ └╥┘
meas: 2/══════════════════════════════╩══╩═
                                      0  1 
0 1
{'01': 1024}
        ┌───┐      ░ ┌───┐ ░      ┌───┐ ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤ Z ├─░───■──┤ H ├─░─┤M├───
        └───┘┌─┴─┐ ░ └───┘ ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░───────░─┤ X ├──────░──╫─┤M├
             └───┘ ░       ░ └───┘      ░  ║ └╥┘
meas: 2/═══════════════════════════════════╩══╩═
                                           0  1 
1 0
{'10': 1024}
        ┌───┐      ░ ┌───┐ ░      ┌───┐ ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤ X ├─░───■──┤ H ├─░─┤M├───
        └───┘┌─┴─┐ ░ └───┘ ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░───────░─┤ X ├──────░──╫─┤M├
             └───┘ ░       ░ └───┘      ░  ║ └╥┘
meas: 2/═══════════════════════════════════╩══╩═
    

In [2]:
def m(i,j,s):
    qc = QuantumCircuit(3)
    qc.h(0)
    qc.cx(0,1)
    qc.cx(0,2)

    if i:
        qc.y(0)
    if j:
        qc.z(0)     
    if s:
        qc.x(1)
  
    qc.cx(0,2)
    qc.cx(0,1)
    qc.h(0)
    qc.measure_all()
    return qc


for i in range(2):
    for j in range(2):
        for s in range(2):
            qc = m(i,j,s)
            #print(qc.draw())
            result = sim.run(qc).result()
            print(result.get_counts())

{'000': 1024}
{'010': 1024}
{'001': 1024}
{'011': 1024}
{'111': 1024}
{'101': 1024}
{'110': 1024}
{'100': 1024}


## Expectation and Standard Deviation

The exposition this year began with observables. However, there was not so much focus on expectation.

Expectation came up as a result of the quantum innerproduct, where the numeric result was incoded in the bias of a coin. However, for me it was an important reaization that, since our experiements only sample an outcome (to use somewhat faulty probabilistic notions) only what is known is the expectation. This is unless the standard deviation is zero, in which case we can disprove a state. 

It is also important to introduce the commutation to explain why qubits can be measured individually. For the measurement of one of a pair of qubits, the observable is,
$$
O_0 =
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{bmatrix} 
$$
and 
$$
O_1=
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 \\
\end{bmatrix} 
$$

and $[O_0,O_1]=0$.

A final result, that came up as a surprise, is this recursion formula for half angles, which leads to 
very pretty cascades of "two plus square roots", ending in $\sqrt{2}$ or $\sqrt{3}$, depending on whether
there are half-half angles of $\pi/4$ or $\pi/3$.

$$
2\cos\theta = \sqrt{2 + 2 \cos 2\theta}
$$

In [3]:
import math

# a good way to write the 1/2 angle formula for recurrence
# 2 cos(theta) = sqrt(2 + 2 cos(2 theta))

c = 1/math.sqrt(2)
d = 4
for i in range(6):
    assert math.isclose(c,math.cos(math.pi/d))
    print(f'cos(pi/{d}):\t{c:.4f}')
    c = (1/2) * math.sqrt(2+2*c)
    d = d*2
    pass
    

cos(pi/4):	0.7071
cos(pi/8):	0.9239
cos(pi/16):	0.9808
cos(pi/32):	0.9952
cos(pi/64):	0.9988
cos(pi/128):	0.9997


## End